In [2]:
import jieba;
jieba.enable_paddle();
#ya,I think jieba are kinda magical

import copy;
import math;
import json;
import requests;
from random import randrange
from bs4 import BeautifulSoup;



things={};
with open("./output415-Copy1.txt",encoding='utf-8') as ifile:
    things=json.loads(ifile.read());
print("done!");

Paddle enabled successfully......


done!


In [10]:
print(things["data"][1]);

{'name': '蝙蝠俠', 'link': 'https://movies.yahoo.com.tw/movieinfo_main/%E8%9D%99%E8%9D%A0%E4%BF%A0-the-batman-10904', 'when': '2022-03-04', 'desc': '《蝙蝠俠》將聚焦於一個年輕版本的布魯斯韋恩，並從中檢視他在面罩之下的靈魂。羅伯派汀森將在新版《蝙蝠俠》中飾演這位DC超級英雄，電影是由《猩球崛起：終極決戰》導演麥特李維斯（Matt Reeves）所執導。', 'class': ['動作', '犯罪']}


In [40]:

def scan_doc(inname,inlist):
    doc={
        "name":inname,
        "total_word_count":0,
        "stat":{}# word -> {count,idf}
    };
    
    #scan thorogh list
    for word in inlist:
        if word in doc["stat"]:
            doc["stat"][word]["count"]+=1;
        else:
            doc["stat"][word]={};
            doc["stat"][word]["name"]=word;
            doc["stat"][word]["source"]=inname;
            doc["stat"][word]["count"]=1;
        doc["total_word_count"]+=1;
    #end of loop
   
    return doc;
#end of scan_doc

def scan_doc_list(inlist):
    docs={
        "total_doc_count":0,
        "total_word_count":0,
        "stat":{},# word -> {count,idf}
        "doc":{}#line -> doc
    };
    
    #scan every doc in lista
    for line,line_of_list in inlist.items():
        doc=scan_doc(line,line_of_list);
        docs["doc"][line]=doc;
        #merge stat loop
        for word,info in doc["stat"].items():
            if word in docs["stat"]:
                docs["stat"][word]["count"]+=doc["stat"][word]["count"];
                docs["stat"][word]["doc_count"]+=1;
            else:
                docs["stat"][word]={};
                docs["stat"][word]["name"]=word;
                docs["stat"][word]["count"]=copy.deepcopy(doc["stat"][word]["count"]);
                docs["stat"][word]["doc_count"]=1;
        #end of merge loop
        docs["total_doc_count"]+=1;
        docs["total_word_count"]+=doc["total_word_count"];
    #end of scan
    
    for word,info in docs["stat"].items():
        info["idf"]=math.log((docs["total_doc_count"])/(info["doc_count"]),10);
    #end of idf cal
    
    for word,doc in docs["doc"].items():
        for word,info in doc["stat"].items():
            tr=(info["count"])/(doc["total_word_count"]);
            idf=docs["stat"][word]["idf"];
            info["tfidf"]=(tr)*(idf);
        #end of a word
    #end of a doc
    return docs;
#end of scan_doc_list


class prediction:
    def __init__(self,indataset):
        self.vector={};#each contain word,index and their idf value
        self.map=[];#each contain list of idf,and class
        
        doc_list={};
        for i,entry in enumerate(indataset):
            line=entry["name"]+entry["desc"];
            cut_list=list(jieba.cut(entry["name"]))+list(jieba.cut(entry["desc"]));
            doc_list[line]=cut_list;
        #end of making thing from list
        
        #yes,code resused
        docs=scan_doc_list(doc_list);
        
        #converter them into vector
        for entry in docs["stat"].values():
            self.vector[entry["name"]]={"idf":entry["idf"]};
        
        for entry in docs["doc"].values():
            point={};
            for word,info in entry["stat"].items():
                point[word]=info["tfidf"];
            self.map.append(point);
                
        print(self.map);
        return;
    def 
    #end of __init
#end of prediction

prediction(things["data"][0:1])

SyntaxError: invalid syntax (<ipython-input-40-1d7a9c612680>, line 92)